**Design Patterns** are generic solutions to problems that you encounter as a developer 

Sometimes there are several options for doing something in your code and depending on your situation you want to do something different, e.g. If you have customer support software you might want to change the way support tickets are processed depending on how busy it is; Or if you are building a VR application and depending on VR setup used you want to use a different rendring algorithm without having to change the code.

In [1]:
import string
import random

from typing import List, Callable
from abc import ABC, abstractmethod

### Before

In [2]:
def generate_id(length: int = 8):
    return ''.join(random.choices(string.ascii_uppercase, k=length))


class SupportTicket:
    
    def __init__(self, customer, issue):
        self.id = generate_id()
        self.customer = customer
        self.issue = issue
        
        
class CustomerSupport:
    
    def __init__(self, processing_strategy: str = "fifo"):
        self.tickets = []
        self.processing_strategy = processing_strategy
        
    def create_ticket(self, customer, issue):
        self.tickets.append(SupportTicket(customer, issue))
        
    def process_tickets(self):
        if not self.tickets:
            print("There are no tickets to process.")
            return
        
        if self.processing_strategy == "fifo":
            for ticket in self.tickets:
                self.process_ticket(ticket)
        elif self.processing_strategy == "filo":
            for ticket in reversed(self.tickets):
                self.process_ticket(ticket)
        elif self.processing_strategy == "random":
            tickets_copy = self.tickets.copy()
            random.shuffle(tickets_copy)
            for ticket in tickets_copy:
                self.process_ticket(ticket)
                
    def process_ticket(self, ticket = SupportTicket):
        print("=====================================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("=====================================================\n")
        
        
        
# Create app
app = CustomerSupport("fifo")

# Register a few tickets
app.create_ticket("Sir Will", "My kennel has water leak!")
app.create_ticket("Grumpy Moose", "My kong was not filled with peanut butter")
app.create_ticket("Jumpy Turbo", "I need thick blankets, please help.")

# Process tickets
app.process_tickets()


# Problem in the above code lies in process_tickets method's IF-ELSE statement -
# process_tickets is quite long, not DRY, and if you want to add more strategies
# then it means you need to extend IF-ELSE statement so currently process_tickets 
# has weak cohesion because it is responsible for processing tickets as well as 
# implementing different strategies. Strategy design pattern aims to solve this 
# type of problem where you have a method that tries to do too many things that 
# has to choose between a strategy of doing something differently. In classic 
# Strategy pattern we are creating a class for each of the processing strategies
# and that class as one method that does actual processing. We define the interface
# for this class using an abstract base class 

Processing ticket id: PMTBHLTT
Customer: Sir Will
Issue: My kennel has water leak!

Processing ticket id: KTXZUTZW
Customer: Grumpy Moose
Issue: My kong was not filled with peanut butter

Processing ticket id: RACMHAQC
Customer: Jumpy Turbo
Issue: I need thick blankets, please help.



### After - Class Based

In [3]:
def generate_id(length: int = 8):
    return ''.join(random.choices(string.ascii_uppercase, k=length))


class SupportTicket:
    id: str
    customer: str
    issue: str
    
    def __init__(self, customer, issue):
        self.id = generate_id()
        self.customer = customer
        self.issue = issue
        

class TicketOrderingStragety(ABC):
    
    @abstractmethod
    def create_ordering(self, tickets: List[SupportTicket]) -> List[SupportTicket]:
        pass
    
    
class FIFOOrderingStrategy(TicketOrderingStragety):
    
    def create_ordering(self, tickets: List[SupportTicket]) -> List[SupportTicket]:
        return tickets.copy()
    
    
class FILOOrderingStrategy(TicketOrderingStragety):
    
    def create_ordering(self, tickets: List[SupportTicket]) -> List[SupportTicket]:
        tickets_copy = tickets.copy()
        tickets_copy.reverse()
        return tickets_copy
    
    
class RandomOrderingStrategy(TicketOrderingStragety):
    
    def create_ordering(self, tickets: List[SupportTicket]) -> List[SupportTicket]:
        tickets_copy = tickets.copy()
        random.shuffle(tickets_copy)
        return tickets_copy
    
    
class BlackHoleStrategy(TicketOrderingStragety):
    
    def create_ordering(self, tickets: List[SupportTicket]) -> List[SupportTicket]:
        return []
    
    
class CustomerSupport:
    tickets: List[SupportTicket] = []
    
    def __init__(self, processing_strategy: TicketOrderingStragety):
        self.tickets = []
        self.processing_strategy = processing_strategy
        
    def create_ticket(self, customer, issue):
        self.tickets.append(SupportTicket(customer, issue))
        
    def process_tickets(self):
        ticket_list = self.processing_strategy.create_ordering(self.tickets)
        
        if not ticket_list:
            print("There are no tickets to process.")
            return
        
        for ticket in ticket_list:
            self.process_ticket(ticket)
                
    def process_ticket(self, ticket = SupportTicket):
        print("=====================================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("=====================================================\n")
        
# Create app
app = CustomerSupport(RandomOrderingStrategy())

# Register a few tickets
app.create_ticket("Sir Will", "My kennel has water leak!")
app.create_ticket("Grumpy Moose", "My kong was not filled with peanut butter")
app.create_ticket("Jumpy Turbo", "I need thick blankets, please help.")

# Process tickets
app.process_tickets()

Processing ticket id: MGIBCRVU
Customer: Sir Will
Issue: My kennel has water leak!

Processing ticket id: KXXWLEBX
Customer: Jumpy Turbo
Issue: I need thick blankets, please help.

Processing ticket id: XCMUNUVY
Customer: Grumpy Moose
Issue: My kong was not filled with peanut butter



### After - Function Based

In [4]:
def generate_id(length: int = 8):
    return ''.join(random.choices(string.ascii_uppercase, k=length))


class SupportTicket:
    id: str
    customer: str
    issue: str
    
    def __init__(self, customer, issue):
        self.id = generate_id()
        self.customer = customer
        self.issue = issue
        
        
def fifo_ordering(tickets: List[SupportTicket]) -> List[SupportTicket]:
    return tickets.copy()


def filo_ordering(tickets: List[SupportTicket]) -> List[SupportTicket]:
    tickets_copy = tickets.copy()
    tickets_copy.reverse()
    return tickets_copy


def random_ordering(tickets: List[SupportTicket]) -> List[SupportTicket]:
    tickets_copy = tickets.copy()
    random.shuffle(tickets_copy)
    return tickets_copy


def black_hole_ordering(tickets: List[SupportTicket]) -> List[SupportTicket]:
    return []


class CustomerSupport:
    tickets: List[SupportTicket] = []
    
    def __init__(self):
        self.tickets = []
        
    def create_ticket(self, customer, issue):
        self.tickets.append(SupportTicket(customer, issue))
        
    def process_tickets(self, ordering: Callable[[List[SupportTicket]], List[SupportTicket]]):
        ticket_list = ordering(self.tickets)
        
        if not ticket_list:
            print("There are no tickets to process.")
            return
        
        for ticket in ticket_list:
            self.process_ticket(ticket)
                
    def process_ticket(self, ticket = SupportTicket):
        print("=====================================================")
        print(f"Processing ticket id: {ticket.id}")
        print(f"Customer: {ticket.customer}")
        print(f"Issue: {ticket.issue}")
        print("=====================================================\n")
        
# Create app
app = CustomerSupport()

# Register a few tickets
app.create_ticket("Sir Will", "My kennel has water leak!")
app.create_ticket("Grumpy Moose", "My kong was not filled with peanut butter")
app.create_ticket("Jumpy Turbo", "I need thick blankets, please help.")

# Process tickets
app.process_tickets(fifo_ordering)

Processing ticket id: GWAVGYVI
Customer: Sir Will
Issue: My kennel has water leak!

Processing ticket id: UJLWKUCL
Customer: Grumpy Moose
Issue: My kong was not filled with peanut butter

Processing ticket id: KPHJRPUJ
Customer: Jumpy Turbo
Issue: I need thick blankets, please help.

